{% include toc title = "In This Lesson" icon="file-text" %}

<div class='notice--success' markdown="1">

## <i class="fa fa-graduation-cap" aria-hidden="true"></i> Learning Objectives

After completing this tutorial, you will be able to:

* Extract geospatial (x,y) coordinate information embedded within a `JSON` hierarchical data structure.
* Convert data imported in `JSON` format into a `Geopandas` `DataFrame`.
* Create a map of geospatial data.


## <i class="fa fa-check-square-o fa-2" aria-hidden="true"></i> What You Need

You will need a computer with internet access to complete this lesson.

</div>

In this lesson, you work with `JSON` data accessed via the Colorado information warehouse. The data will contain geospatial information nested within it that will allow us to create a map of the data.

## Working with Geospatial Data

Check out the map <a href="https://data.colorado.gov/Water/DWR-Current-Surface-Water-Conditions-Map-Statewide/j5pc-4t32" target="_blank"> Colorado DWR Current Surface Water Conditions map</a>. 

Remember from the previous lesson, APIs can be used for many different things. Web developers (people who program and create web sites and cool applications) can use APIs to create user friendly interfaces - like the map in the previous example that allows us to look at and interact with data. These APIs are similar to, if not the same as, the ones that you often use to access data in `Python`.

In this lesson, you will access the data used to create the map at the link above using `Python`.

* The data that you will use are located here: <a href="https://data.colorado.gov/resource/j5pc-4t32.json" target="_blank">View JSON format data used to create surface water map.</a>
* And you can learn more about the data here: <a href="https://data.colorado.gov/Water/DWR-Current-Surface-Water-Conditions/4yw9-a5y6" target="_blank">View CO Current water surface </a>.

In [1]:
import requests
import folium
import urllib
from pandas.io.json import json_normalize
import pandas as pd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [2]:
# Get URL
water_base_url = "https://data.colorado.gov/resource/j5pc-4t32.json?"
water_full_url = water_base_url + "station_status=Active" + "&county=BOULDER"

<i class="fa fa-lightbulb-o" aria-hidden="true"></i> **ATTENTION WINDOWS USERS:**
We have noticed a bug where on windows machines, sometimes the https URL doesn't work.
Instead try the same url as above but without the `s` - like this: `water_base_url = "http://data.colorado.gov/resource/j5pc-4t32.json?"` This change has resolved many issues on windows machines so give it a try if you are having problems with the API.
{: .notice--success }

In [3]:
water_full_url

'https://data.colorado.gov/resource/j5pc-4t32.json?station_status=Active&county=BOULDER'

In [4]:
data = requests.get(water_full_url)

In [5]:
type(data.json())

list

Remember that the JSON structure supports hierarchical data and can be NESTED. If you look at the structure of the `.json` file below, you can see that the location object, is nested with three sub objects:

* latitude
* longitude
* needs_recoding

Since `data.json()` is a `list` you can print out just the first few items of the list to look at your data as a sanity check.

In [6]:
data.json()[:2]

[{'station_name': 'ZWECK AND TURNER DITCH',
  'div': '1',
  'location': {'latitude': '40.185033',
   'needs_recoding': False,
   'longitude': '-105.185789'},
  'dwr_abbrev': 'ZWETURCO',
  'data_source': 'Cooperative Program of CDWR, NCWCD & SVLHWCD',
  'amount': '0.14',
  'station_type': 'Diversion',
  'wd': '5',
  'http_linkage': {'url': 'https://dwr.state.co.us/SurfaceWater/data/detail_graph.aspx?ID=ZWETURCO&MTYPE=DISCHRG'},
  'date_time': '2020-03-23T14:00:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'stage': '0.05',
  'station_status': 'Active'},
 {'station_name': 'SAINT VRAIN CREEK AT LYONS, CO',
  'div': '1',
  'location': {'latitude': '40.220702',
   'needs_recoding': False,
   'longitude': '-105.26349'},
  'dwr_abbrev': 'SVCLYOCO',
  'data_source': 'Co. Division of Water Resources',
  'usgs_station_id': '06724000',
  'amount': '37.90',
  'station_type': 'Stream',
  'wd': '5',
  'http_linkage': {'url': 'https://dwr.state.co.us/SurfaceWater/data/detail_graph.aspx?ID

## Convert JSON to Pandas DataFrame

Now that you have pulled down the data from the website, you have it in the `JSON` format. For the next step, you will use the `json_normalize()` function from the `Pandas` library to convert this data into a `Pandas DataFrame.` 

This function helps organize and flatten data into a semi-structed table. To learn more, check out the <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html" target="_blank">documentation</a>! 

In [7]:
from pandas.io.json import json_normalize

In [8]:
result = json_normalize(data.json())

/Users/leahwasser/miniconda3/envs/earth-analytics-python/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [9]:
result.head()

,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,stage,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,usgs_station_id,flag
0,ZWECK AND TURNER DITCH,1,ZWETURCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.14,Diversion,5,2020-03-23T14:00:00.000,BOULDER,DISCHRG,0.05,Active,40.185033,False,-105.185789,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN
1,"SAINT VRAIN CREEK AT LYONS, CO",1,SVCLYOCO,Co. Division of Water Resources,37.90,Stream,5,2020-03-23T14:15:00.000,BOULDER,DISCHRG,2.96,Active,40.220702,False,-105.26349,https://dwr.state.co.us/SurfaceWater/data/deta...,06724000,NaN
2,OLIGARCHY DITCH DIVERSION,1,OLIDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",6.68,Diversion,5,2020-03-23T12:45:00.000,BOULDER,DISCHRG,0.33,Active,40.196422,False,-105.206592,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN
3,SOUTH BOULDER CREEK DIVERSION NEAR ELDORADO SP...,1,BOSDELCO,Co. Division of Water Resources,0.00,Diversion,6,2020-03-23T14:30:00.000,BOULDER,DISCHRG,-1.70,Active,39.931813,False,-105.308432,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN
4,CLOUGH AND TRUE DITCH,1,CLODITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.00,Diversion,5,2020-03-23T14:00:00.000,BOULDER,DISCHRG,NaN,Active,40.193758,False,-105.21039,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN


In [10]:
type(result)

pandas.core.frame.DataFrame

In [11]:
result.columns

Index(['station_name', 'div', 'dwr_abbrev', 'data_source', 'amount',
       'station_type', 'wd', 'date_time', 'county', 'variable', 'stage',
       'station_status', 'location.latitude', 'location.needs_recoding',
       'location.longitude', 'http_linkage.url', 'usgs_station_id', 'flag'],
      dtype='object')

## Data Cleaning for Visualization

Now you can clean up the data. Notice that your longitude and latitude values are stored as strings. Do you think can create a map if these values are stored as strings?

In [12]:
result['location.latitude'][0]

'40.185033'

You can convert the strings to type `float` as follows. 

In [13]:
result['location.latitude'] = result['location.latitude'].astype(float)

In [14]:
result['location.latitude'][0]

40.185033

In [15]:
result['location.longitude'] = result['location.longitude'].astype(float)

In [16]:
result['location.longitude'][0]

-105.185789

Now that you have numeric values for mapping, make sure that are are no missing values. 

In [17]:
result.shape

(49, 18)

In [18]:
result['location.longitude'].isna().any()

False

In [19]:
result['location.latitude'].isna().any()

False

There are no `nan` values in this data. However, if there were, you could remove rows where a column has a `nan` value in a specific column with the following: 
`result_nonan = result.dropna(subset=['location.longitude', 'location.latitude'])`

## Data Visualization 

You will use the `folium` package to visualize the data. One approach you could take would be to convert your `Pandas DataFrame` to a `Geopandas DataFrame` for easy mapping.

In [20]:
geometry = [Point(xy) for xy in zip(result['location.longitude'], result['location.latitude'])]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(result, crs=crs, geometry=geometry)

In [21]:
gdf.head()

,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,stage,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,usgs_station_id,flag,geometry
0,ZWECK AND TURNER DITCH,1,ZWETURCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.14,Diversion,5,2020-03-23T14:00:00.000,BOULDER,DISCHRG,0.05,Active,40.185033,False,-105.185789,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN,POINT (-105.18579 40.18503)
1,"SAINT VRAIN CREEK AT LYONS, CO",1,SVCLYOCO,Co. Division of Water Resources,37.90,Stream,5,2020-03-23T14:15:00.000,BOULDER,DISCHRG,2.96,Active,40.220702,False,-105.263490,https://dwr.state.co.us/SurfaceWater/data/deta...,06724000,NaN,POINT (-105.26349 40.22070)
2,OLIGARCHY DITCH DIVERSION,1,OLIDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",6.68,Diversion,5,2020-03-23T12:45:00.000,BOULDER,DISCHRG,0.33,Active,40.196422,False,-105.206592,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN,POINT (-105.20659 40.19642)
3,SOUTH BOULDER CREEK DIVERSION NEAR ELDORADO SP...,1,BOSDELCO,Co. Division of Water Resources,0.00,Diversion,6,2020-03-23T14:30:00.000,BOULDER,DISCHRG,-1.70,Active,39.931813,False,-105.308432,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN,POINT (-105.30843 39.93181)
4,CLOUGH AND TRUE DITCH,1,CLODITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.00,Diversion,5,2020-03-23T14:00:00.000,BOULDER,DISCHRG,NaN,Active,40.193758,False,-105.210390,https://dwr.state.co.us/SurfaceWater/data/deta...,NaN,NaN,POINT (-105.21039 40.19376)


Then, you can plot the data using the folium functions `GeoJson()` and `add_to()` to add the data from the `Geopandas DataFrame` to the map object. 

In [22]:
m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(m)

m

/Users/leahwasser/miniconda3/envs/earth-analytics-python/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


Great! You now have an interactive map in your notebook! 

You can also cluster the markers, and add a popup to each marker, so you can give your viewers more information about station: such as its name and the amount of precipitation measured.

For this example below, you will work with the `Pandas DataFrame` you originally created from the `JSON`, instead of the `Geopandas GeoDataFrame`.

In [23]:
# Get the latitude and longitude from result as a list
locations = result[['location.latitude', 'location.longitude']]
coords = locations.values.tolist()

In [24]:
from folium.plugins import MarkerCluster

m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(m)

for point in range(0, len(coords)):
    folium.Marker(location = coords[point], popup= 'Name: ' + result['station_name'][point] + ' ' + 'Precip: ' + str(result['amount'][point])).add_to(marker_cluster)

m

<div class="notice--info" markdown="1">

## Additional Resources

* <a href="https://blog.datafiniti.co/4-reasons-you-should-use-json-instead-of-csv-2cac362f1943" target="_blank">Why you should use JSON instead of csv.</a>


* <a href="https://www.w3schools.com/js/js_json_intro.asp" target="_blank">W3schools JSON intro </a>

</div>